# Read SQL table with Pandas

In [ ]:
import pandas as pd
import numpy as np
from datetime import date, datetime
from dateutil import parser
from pytz import timezone
from dateutil import parser

from sqlalchemy.orm import sessionmaker
from scrapy.exceptions import DropItem
from FiScrape.models import Article, Tag, db_connect, create_table, Topic, Source, Author, SnipBlob, Blob, SnipVader, Vader #  create_output_table
# from FiScrape.spiders.FiSpider import query
import logging
# pip install -U textblob
from textblob import TextBlob
# pip install vaderSentiment
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

In [ ]:
# con = 'sqlite:///FiScrape_01.db'
con = 'sqlite:///FiScrape.db'

In [ ]:
df = pd.read_sql_table("article", con, schema=None, index_col='id', coerce_float=True, parse_dates='published_date', chunksize=None) # columns=['headline, standfirst'],
df.head()

In [ ]:
search = 'headline'
text = 'Bitcoin mining producing tonnes of waste'
output = 'content'
# output = 'article_link'

article = df.loc[df[search].astype(str) == text]
for line in article[output]:
    print (line)


In [ ]:
type(article[output].values)
article[output].values

In [ ]:
for line in article[output].values:
    body = ''.join(line)
print (body)

In [ ]:
body

In [ ]:
for content in df['content']:
    print (content)

In [ ]:
for standfirst in df['standfirst']:
    print (standfirst)

In [ ]:
for article_link in df['article_link']:
    print (article_link)

In [ ]:
# for row in df['standfirst']:
#     print (row)

In [ ]:
# # for row in df['standfirst']:
# #     row = row.strip().replace("  ", " ")
# #     row = ' '.join(row.split())
# #     row = row.replace(' .', '. ').replace(' ,', ',')
# #     row = row + '.'
# #     row = row.replace('..', '.').replace('.', '. ').strip()
# #     print (row)

# df['standfirst'] = df['standfirst'].apply(lambda x: x.strip().replace("  ", " ").replace('  ', ' ').replace('  ', ' '))
# df['standfirst'] = df['standfirst'].apply(lambda x: ' '.join(x.split()))
# df['standfirst'] = df['standfirst'].apply(lambda x: x.replace(' .', '. ').replace('.', '. ').replace(' ,', ','))
# df['standfirst'] = df['standfirst'].apply(lambda x: x + '...').replace(' ...', '...').replace('......', '...')
# df['standfirst'] = df['standfirst'].apply(lambda x: x.replace('?...', '?').replace('!...', '!').replace('-...', '-'))
# df['standfirst'] = df['standfirst'].apply(lambda x: x.replace('. .', '.').strip())
# df['standfirst'] = df['standfirst'].apply(lambda x: x.replace(':', ': ').replace(':  ', ': ').replace(' ;', ';'))
# df['standfirst'] = df['standfirst'].apply(lambda x: x.replace('...', '... ').replace(' .', '... ').strip())
# df['standfirst'] = df['standfirst'].apply(lambda x: x.replace('“ ', '“').replace(' ”','”').replace(" ’", "’").replace(" ’", "’"))
# df['standfirst'] = df['standfirst'].apply(lambda x: x.strip().replace("  ", " ").replace('  ', ' ').replace('  ', ' ')) 
# df['standfirst'] = df['standfirst'].apply(lambda x: x.replace(' ... ..', '...').replace('......', '...').replace('..... ..', '...'))
# df['standfirst'] = df['standfirst'].apply(lambda x: x.replace('......', '...').replace('....', '...'))
# df['standfirst'] = df['standfirst'].apply(lambda x: x.replace('  ', ' '))

# for row in df['standfirst']:
#     print (row)

In [ ]:
id_ = 57
snippet = df['headline'][id_] +' - ...'+ df['standfirst'][id_]
print (snippet)

In [ ]:
engine = db_connect()
create_table(engine)
Session = sessionmaker(bind=engine)
logging.info("****SentimentPipeline: database connected****")

In [ ]:
link ="https://www.ft.com/content/31f7edf7-8e05-46e1-8b13-061532f8db5f"
session = Session()
exist_article = session.query(Article).filter_by(article_link = link).first()
import warnings
warnings.filterwarnings('ignore')
exist_article

In [ ]:
# exist_article.snip_blob['subjectivity']
head = exist_article.headline
sf = exist_article.standfirst
text = ' — ...'.join([head,sf])
print (text)

In [ ]:
query = 'Bitcoin'

class SentimentPipeline(object):

    def __init__(self):
        """
        Initializes database connection and sessionmaker.
        Creates tables.
        """
        engine = db_connect()
        create_table(engine)
        self.Session = sessionmaker(bind=engine)
        logging.info("****SentimentPipeline: database connected****")

    def get_sentiment(self, item, spider):
        """
        Calculates and stores sentiment scores for the snippet (headline & standfirst), and the article body, if it was scraped.
        """
        session = self.Session()
        exist_article = session.query(Article).filter_by(article_link = item["article_link"]).first()
        session.close()
        if exist_article is not None:  # the current article exists
            head = exist_article.headline
            sf = exist_article.standfirst
            text = ' — ...'.join([head,sf])

            article = exist_article
            snip_blob = SnipBlob(name=article.article_link)

            subjectivity_score = self.get_subjectivity(text)
            snip_blob.subjectivity = subjectivity_score
            polarity_score = self.get_polarity(text)
            snip_blob.polarity = polarity_score

            snip_vader = SnipVader(name=article.article_link)
            SIA = 0
            SIA = self.get_SIA(text)
            compound = (SIA['compound'])
            neg = (SIA['neg'])
            neu = (SIA['neu'])
            pos = (SIA['pos'])
            snip_vader.compound = compound
            snip_vader.negative = neg
            snip_vader.neutral = neu
            snip_vader.negative = neg
            
        try:
            # session.add(article)
            session.commit()

        except:
            session.rollback()
            raise

        finally:
            session.close()

    # Get the subjectivity
    def get_subjectivity(self, text):
        """
        Returns a subjectivity score between 0 and 1.
        0 is objective and 1 is subjective.
        """
        return TextBlob(text).sentiment.subjectivity

    # Get the polarity
    def get_polarity(self, text):
        """
        Returns a polarity score between -1 and 1.
        -1 is negative sentiment and 1 is positive sentiment.
        """
        return TextBlob(text).sentiment.polarity

    # Get the sentiment scores
    def get_SIA(text):
        sia = SentimentIntensityAnalyzer()
        sentiment =sia.polarity_scores(text)
        return sentiment

In [ ]:
# Get the polarity
def get_polarity(text):
    """
    Returns a polarity score between -1 and 1.
    -1 is negative sentiment and 1 is positive sentiment.
    """
    return TextBlob(text).sentiment.polarity

# Get the subjectivity
def get_subjectivity(text):
    """
    Returns a subjectivity score between 0 and 1.
    0 is objective and 1 is subjective.
    """
    return TextBlob(text).sentiment.subjectivity

def get_SIA(text):
    sia = SentimentIntensityAnalyzer()
    sentiment =sia.polarity_scores(text)
    return sentiment

In [ ]:
# text_inp = "Dalio warns regulators will ‘kill’ bitcoin if it becomes too successful Wood’s investment firm has unveiled plans for a bitcoin exchange traded fund, although it is yet to receive regulatory approval...."
text_inp = 'He said he will "kill" her slowly.'
# text_inp = snippet
print (text_inp)

In [ ]:
# text_inp = exist_article.standfirst
# text_inp = text
# text_inp = "China expands crackdown by declaring all crypto activities ‘illegal’"
# text_inp = "...have continued to invest in cryptocurrency using foreign platforms. The price of bitcoin fell more than 8 per cent immediately after the announcement, dropping to"

polarity = get_polarity(text_inp)
subj = get_subjectivity(text_inp)
SIA = get_SIA(text_inp)
print (text_inp)
print ("Polarity:", polarity, "Subjectivity:", subj)
print(SIA)

In [ ]:
polarity = get_polarity(text_inp)
subj = get_subjectivity(text_inp)
SIA = get_SIA(text_inp)
print (text_inp)
print ("Polarity:", polarity, "Subjectivity:", subj)
print(SIA)

In [ ]:
neg = SIA['neg']
neg

In [ ]:
’ ''

In [ ]:
exist_article.snip_blob
